In [92]:
import json
from llamaapi import LlamaAPI

description = "A 50 square meters apartment for rent in Campanar, that costs 500 euros per month and has two bedrooms, one bathroom and terrace."
description = "A 100 square meter apartment for sale in Cortes Valencianas, with three bedrooms, two bathrooms, that is bright and has a balcony."
#description = "Un piso de alquiler en Campanar que cueste menos de 500 euros con dos habitaciones, al menos un baño y terraza"
# Initialize the SDK
llama = LlamaAPI("LL-AtV042dq78TAmTiO12qRWx83rJR3G4X6d2UDXo8OAbOFwwDK0LjItrqnnXreYEdC")

# Build the API request
api_request_json = {
    'model': "llama-13b-chat",
    "functions": [
        {
            "name": "get_apartment_info",
            "description": "Get information on the characteristics of an apartment",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location of the apartment, e.g. Betera",
                    },
                    "rooms": {
                        "type": "number",
                        "description": "The number of rooms in the apartment",
                    },
                    "bathrooms": {"type": "number",
                        "description": "The number of bathrooms in the apartment",
                    },
                    "price": {"type": "number",
                        "description": "The price of the apartment",
                    },
                    "surface": {"type": "number",
                        "description": "The surface of the apartment in square meters",
                    },
                    "contract": {"type": "string",
                        "description": "The type of apartment contract, e.g. rent, sale",
                    }
                },
            },
            "required": ["location", "rooms", "bathrooms", "price", "surface", "contract"],
        }
    ],
    "function_call": {"name":"get_apartment_info"},
    "messages": [
        {"role": "user", "content": "Get the characteristics of an apartment from the following description: " + description},
        #{"role": "system", "content": "Return a JSON object with the characteristics of the apartment specified in the function call."},
    ],
}

In [93]:
# Execute the Request
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1714647380,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 267,
    "completion_tokens": 34,
    "total_tokens": 301
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_apartment_info",
          "arguments": {
            "location": "Cortes Valencianas",
            "rooms": 3,
            "bathrooms": 2,
            "surface": 100
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


In [89]:
import pandas as pd

choices = response.json().get("choices", [])
if choices:
    function_call = choices[0].get("message", {}).get("function_call", {})
    arguments = function_call.get("arguments", {})

    # Construir el DataFrame con las características
    data = {
        'location': [arguments.get('location', 'NA')],
        'rooms': [arguments.get('rooms', 'NA')],
        'bathrooms': [arguments.get('bathrooms', 'NA')],
        'price': [arguments.get('price', 'NA')],
        'surface': [arguments.get('surface', 'NA')],
        'contract': [arguments.get('contract', 'NA')],
    }

    # Crear un DataFrame
    df = pd.DataFrame(data)
    print(df)
else:
    print("No se encontraron resultados.")

             location  rooms  bathrooms price  surface contract
0  Cortes Valencianas      3          2    NA      100       NA


In [94]:
import json
import requests

# Definir los datos de la solicitud
description = "A 50 square meters apartment for rent in Campanar, and has two bedrooms, one bathroom and terrace."
llama_call = {
  "model": "llama3",
  "prompt": "Provide a JSON object with the fields location, rooms, bathrooms, price, surface and contract, that correspond to the location of an apartment, its number of rooms, number of bathrooms, price, surface and contract type, respectively. The contract type can be 'rent' or 'sale'. Extract the information from the following description:" + description,
  "format": "json",
  "stream": False
}
# Realizar la solicitud HTTP
url = "http://localhost:11434/api/generate"
response = requests.post(url, json=llama_call)

# Obtener el campo "response" y convertirlo en un objeto JSON
response_json = json.loads(response.json()["response"])

# Guardar el objeto JSON con los datos de la solicitud
data = json.dumps(response_json, indent=2)

In [95]:
import pandas as pd

# Convierte el objeto JSON en un DataFrame
df = pd.DataFrame.from_dict(response_json, orient='index')

# Transpone el DataFrame para tener los datos en las filas
df = df.transpose()

# Ahora puedes imprimir o manipular el DataFrame según tus necesidades
print(df)

   location rooms bathrooms price surface contract
0  Campanar     2         1  None      50     rent
